In [20]:
# -----------------------------
# Step 1: Imports
# -----------------------------
import pandas as pd
import faiss
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import gradio as gr

In [21]:
# -----------------------------
# Step 2: Load Dataset (your FAQ dataset in CSV)
# -----------------------------
faq_df = pd.read_csv("C://Users//ramiy//Downloads//college_faq_cleaned.csv")  # columns: question, answer, intent
# Expecting columns: "Question", "Answer"
print(faq_df.head())


            question                                             answer  \
0              Hello  Hello! Welcome to our college. How can I assis...   
1                 Hi                      Hi there! How can I help you?   
2       Good morning  Good morning! What would you like to know abou...   
3             Thanks                 You're welcome! Glad I could help.   
4  Thank you so much          Anytime! Feel free to ask more questions.   

     intent  
0  greeting  
1  greeting  
2  greeting  
3  thankyou  
4  thankyou  


In [22]:
print(faq_df.columns)


Index(['question', 'answer', 'intent'], dtype='object')


In [23]:
faq_questions = faq_df["question"].tolist()
faq_answers = faq_df["answer"].tolist()

In [24]:
# -----------------------------
# Step 3: Encode Questions into Vectors (for Retrieval)
# -----------------------------
embedder = SentenceTransformer("all-MiniLM-L6-v2")  # small + fast
faq_embeddings = embedder.encode(faq_questions, convert_to_numpy=True)

C:\Users\ramiy\.conda\envs\hf_EduGuide\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [25]:
# Build FAISS index
dim = faq_embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(faq_embeddings)


In [26]:
# -----------------------------
# Step 4: Load Generative Model (Fallback for unseen queries)
# -----------------------------
gen_model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(gen_model_name)
gen_model = AutoModelForSeq2SeqLM.from_pretrained(gen_model_name)
generator = pipeline("text2text-generation", model=gen_model, tokenizer=tokenizer)


In [27]:
# -----------------------------
# Step 5: Chatbot Function
# -----------------------------
def eduguide_fn(user_query, history, state):
    # Encode query
    query_emb = embedder.encode([user_query], convert_to_numpy=True)
    
    # Retrieve top-1 match
    D, I = index.search(query_emb, 1)  # distance, index
    best_idx = I[0][0]
    best_score = D[0][0]

    # Threshold → if retrieval confidence is low, use generative
    if best_score < 50:  
        answer = faq_answers[best_idx]
    else:
        # Fallback to generative model
        prompt = f"Answer this question related to college admissions/facilities/courses:\n{user_query}"
        gen_output = generator(prompt, max_length=128, num_return_sequences=1)
        answer = gen_output[0]['generated_text']

    return answer, state

In [28]:
# -----------------------------
# Step 6: Gradio UI
# -----------------------------
with gr.Blocks(title="EduGuide: College Admissions & Career Chatbot") as demo:
    gr.Markdown("## CollegeCare Bot — College Admissions & Career Chatbot")
    gr.Markdown("Ask about **courses, admissions, hostel, placements, scholarships, or campus life**")

    chat = gr.Chatbot(height=500)
    state = gr.State({"last_intent": None, "last_entities": set()})
    txt = gr.Textbox(label="Type your question…", placeholder="e.g., What is the fee structure for B.Tech?")
    submit = gr.Button("Submit", variant="primary")
    clear = gr.Button("Clear Chat")

    def _respond(m, h, s):
        reply, s = eduguide_fn(m, h, s)
        return h + [(m, reply)], s, ""

    txt.submit(_respond, [txt, chat, state], [chat, state, txt])
    submit.click(_respond, [txt, chat, state], [chat, state, txt])

    def _clear():
        return [], {"last_intent": None, "last_entities": set()}

    clear.click(_clear, None, [chat, state], queue=False)

demo.queue().launch()


C:\Users\ramiy\AppData\Local\Temp\ipykernel_17628\3705370895.py:8: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat = gr.Chatbot(height=500)


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
